# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('../data/sp_500_stocks.csv')
from secrets_api import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2721897023296,
 'week52high': 184.89,
 'week52low': 129.41,
 'week52highSplitAdjustOnly': 186.74,
 'week52lowSplitAdjustOnly': 134.2,
 'week52change': 0.10780814661517067,
 'sharesOutstanding': 16638291905,
 'float': 0,
 'avg10Volume': 64200227,
 'avg30Volume': 67100172,
 'day200MovingAvg': 160.89,
 'day50MovingAvg': 163.95,
 'employees': 153499,
 'ttmEPS': 6.11,
 'ttmDividendRate': 0.91407649782911,
 'dividendYield': 0.00571390437695087,
 'nextDividendDate': '',
 'exDividendDate': '2022-08-04',
 'nextEarningsDate': '2022-10-20',
 'peRatio': 27.22675736069981,
 'beta': 1.257594636551063,
 'maxChangePercent': 64.934137078274,
 'year5ChangePercent': 3.389626624359727,
 'year2ChangePercent': 0.31669995152120256,
 'year1ChangePercent': 0.12089504521545348,
 'ytdChangePercent': -0.07781307806350919,
 'month6ChangePercent': -0.004710036847747606,
 'month3ChangePercent': 0.14567411284893858,
 'month1ChangePercent': 0.08409033112750201,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.12089504521545348

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for symbol in symbol_groups:
    symbol_strings.append(','.join(symbol))

print(symbol_strings)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            df = df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ], 
            index=my_columns),
            ignore_index=True)

print(df)

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/139

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/139

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52554/139

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [ ]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace=True)
print(df)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [ ]:
def get_portfolio_size():
    portfolio_size = None
    invalid_val = True
    while portfolio_size is None or invalid_val:
        portfolio_size = input('How big is your portfolio? ')
        try:
            val = float(portfolio_size)
            invalid_val = False
        except:
            print('Please enter a Number (Int or Float).')
        
    return val

portfolio_size = get_portfolio_size()
print(portfolio_size)

In [10]:
position_size = float(portfolio_size/len(df.index))
for i in range(0, len(df)):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[i, 'Price'])

df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,83.04,2.379537,240
1,355,OXY,75.67,2.153553,264
2,89,CF,120.75,1.762499,165
3,147,DVN,72.82,1.682412,274
4,315,MRO,26.24,1.358007,762
5,42,APA,41.25,1.334536,484
6,109,COP,113.83,1.115499,175
7,159,EOG,129.63,1.007957,154
8,312,MOS,64.07,0.998286,312
9,465,VLO,123.67,0.965423,161


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            hqm_df = hqm_df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ]
            , index=hqm_columns),
            ignore_index=True)
            
hqm_df
    

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_52536/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.05,N/A,-0.252996,N/A,-0.025896,N/A,0.050768,N/A,0.038393,N/A,N/A
1,AAL,14.31,N/A,-0.309858,N/A,-0.214138,N/A,-0.207724,N/A,0.023481,N/A,N/A
2,AAP,178.44,N/A,-0.14918,N/A,-0.144685,N/A,-0.083868,N/A,-0.052388,N/A,N/A
3,AAPL,170.75,N/A,0.118531,N/A,-0.004763,N/A,0.141001,N/A,0.081645,N/A,N/A
4,ABBV,141.69,N/A,0.182988,N/A,-0.072608,N/A,-0.088638,N/A,-0.099358,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,114.20,N/A,-0.132396,N/A,-0.088503,N/A,-0.043122,N/A,-0.04467,N/A,N/A
497,ZBH,109.81,N/A,-0.280296,N/A,-0.154876,N/A,-0.109729,N/A,-0.004926,N/A,N/A
498,ZBRA,317.79,N/A,-0.474105,N/A,-0.263145,N/A,-0.076282,N/A,-0.059309,N/A,N/A
499,ZION,57.67,N/A,0.038086,N/A,-0.195077,N/A,0.008039,N/A,0.080755,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
# Cleaning. I have 'None' values...

for time_period in time_periods:
    hqm_df[f'{time_period} Price Return'] = hqm_df[f'{time_period} Price Return'].apply(lambda x: x if x is not None else 0.0)

    for val in hqm_df[f'{time_period} Price Return']:
        if val is None:
            print('!!')

NameError: name 'time_periods' is not defined

In [ ]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])/100
    
hqm_df

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
        
        hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
        
hqm_df

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True, drop=True)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.050,N/A,-0.252996,N/A,-0.025896,N/A,0.050768,N/A,0.038393,N/A,N/A
1,NFLX,232.840,N/A,-0.596314,N/A,-0.437561,N/A,0.169175,N/A,0.044907,N/A,N/A
2,NVDA,167.000,N/A,-0.272489,N/A,-0.329779,N/A,-0.093125,N/A,-0.016539,N/A,N/A
3,NUE,146.710,N/A,0.219315,N/A,0.089418,N/A,0.061076,N/A,0.144863,N/A,N/A
4,NTRS,96.800,N/A,-0.180257,N/A,-0.184274,N/A,-0.125418,N/A,0.000979,N/A,N/A
5,NTAP,77.980,N/A,-0.108984,N/A,-0.05286,N/A,0.07307,N/A,0.081961,N/A,N/A
6,NSC,256.480,N/A,-0.007897,N/A,-0.019372,N/A,0.075548,N/A,0.040581,N/A,N/A
7,NRG,42.060,N/A,-0.043909,N/A,0.124281,N/A,-0.098419,N/A,0.164483,N/A,N/A
8,NOW,445.110,N/A,-0.30163,N/A,-0.24497,N/A,-0.022513,N/A,0.053018,N/A,N/A
9,NOV,19.080,N/A,0.451001,N/A,0.10921,N/A,-0.054979,N/A,0.234575,N/A,N/A


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_size = get_portfolio_size()

How big is your portfolio? 1000000


In [15]:
position_size = float(portfolio_size/len(hqm_df.index))
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.050,148,-0.252996,N/A,-0.025896,N/A,0.050768,N/A,0.038393,N/A,N/A
1,NFLX,232.840,85,-0.596314,N/A,-0.437561,N/A,0.169175,N/A,0.044907,N/A,N/A
2,NVDA,167.000,119,-0.272489,N/A,-0.329779,N/A,-0.093125,N/A,-0.016539,N/A,N/A
3,NUE,146.710,136,0.219315,N/A,0.089418,N/A,0.061076,N/A,0.144863,N/A,N/A
4,NTRS,96.800,206,-0.180257,N/A,-0.184274,N/A,-0.125418,N/A,0.000979,N/A,N/A
5,NTAP,77.980,256,-0.108984,N/A,-0.05286,N/A,0.07307,N/A,0.081961,N/A,N/A
6,NSC,256.480,77,-0.007897,N/A,-0.019372,N/A,0.075548,N/A,0.040581,N/A,N/A
7,NRG,42.060,475,-0.043909,N/A,0.124281,N/A,-0.098419,N/A,0.164483,N/A,N/A
8,NOW,445.110,44,-0.30163,N/A,-0.24497,N/A,-0.022513,N/A,0.053018,N/A,N/A
9,NOV,19.080,1048,0.451001,N/A,0.10921,N/A,-0.054979,N/A,0.234575,N/A,N/A


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('../output/momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                'A': ['Ticker', string_template],
                'B': ['Price', dollar_template],
                'C': ['Number of Shares to Buy', integer_template],
                'D': ['One-Year Price Return', percent_template],
                'E': ['One-Year Return Percentile', percent_template],
                'F': ['Six-Month Price Return', percent_template],
                'G': ['Six-Month Return Percentile', percent_template],
                'H': ['Three-Month Price Return', percent_template],
                'I': ['Three-Month Return Percentile', percent_template],
                'J': ['One-Month Price Return', percent_template],
                'K': ['One-Month Return Percentile', percent_template],
                'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()